In [1]:
import os
import pandas as pd
import numpy as numpy
from datetime import datetime, timedelta
import yfinance as yf

import matplotlib.pyplot as plt
import plotly.express as px
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns

In [2]:
sp500_etf_dict = {
    'Consumer_Discretionary': 'XLY'
    ,'Consumer_Staples': 'XLP'
    ,'Energy': 'XLE'
    ,'Financial': 'XLF'
    ,'Health Care': 'XLV'
    ,'Industrial': 'XLI'
    ,'Material': 'XLB'
    ,'Technology': 'XLK'
    ,'Utilities': 'XLU'
    ,'SP_500': 'SPY'
}

In [ ]:
search_date_str = input("Enter date (YYYY-MM-DD): ")
print("You entered:", search_date_str)


today_str = datetime.today().strftime('%Y-%m-%d')
# Calculate the start date for the earliest period (20 years ago), making it timezone-aware
start_date_str = (datetime.now(pytz.UTC) - timedelta(days=365*30)).strftime('%Y-%m-%d')

today_dt = datetime.now(pytz.UTC)
search_date_dt = datetime.strptime(search_date_str, "%Y-%m-%d").replace(tzinfo=pytz.UTC)


# Periods to fetch the data for
periods = [
    today_dt
    ,search_date_dt

           ]

# DataFrame to store the results
df_closing_prices = pd.DataFrame(index=sp500_etf_dict.keys(), columns=periods)

# Function to get the closing price for specific periods
def get_historical_closing_prices(ticker, start_date_str):
    
    etf = yf.Ticker(ticker)
    hist = etf.history(start=start_date_str)
    
    return hist['Close']


def find_nearest_date(target_date, dates_list):
    """
    Finds and returns the date from dates_list that is closest to target_date.

    Parameters:
    - target_date: The target datetime object you're trying to find the closest match for.
    - dates_list: A list of datetime objects to search within.

    Returns:
    - closest_date: The date from dates_list that is closest to the target_date.
    """

    date_diff_dict = dict(
                        zip(
                            [abs(target_date - i).days for i in dates_list]
                            ,dates_list
                            )
                        )
    # Retrieve the value corresponding to the smallest key
    neaerest_date = date_diff_dict[min(date_diff_dict.keys())].strftime('%Y-%m-%d')


    return neaerest_date




# Fetch and process the data for each ETF
for sector, ticker in sp500_etf_dict.items():
    closing_prices = get_historical_closing_prices(ticker, start_date_str)
    for period in periods:
        if period == today_dt:
            # Get the most recent closing price
            period_date = closing_prices.index[-1]
        if period == search_date_dt:
            period_date = search_date_dt

        # Find the closest date index to our target period_date
        period_date_str = period_date.strftime('%Y-%m-%d')
        if period_date_str in closing_prices.index.strftime('%Y-%m-%d'):
            closest_date = period_date_str
        else:
            closest_date = find_nearest_date(period_date, closing_prices.index)
        # Store the corresponding closing price
        df_closing_prices.at[sector, period] = round(closing_prices.loc[closest_date], 2)
        df_closing_prices['ticker'] = sp500_etf_dict.values()

# df_closing_prices.columns.values[0] = today_str
# df_closing_prices.columns.values[1] = search_date_str

# Display the DataFrame
df_closing_prices

You entered: 2015-01-28


: 